### Imports

In [1]:
# Other imports
import numpy as np
import pandas as pd
import statistics as st
import urllib
import zipfile

# Scikit learn
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


### Datasets

Vamos a cargar los Datasets del primer ejercicio de SVM, para resumir un poco el contenido de estos tenemos que recordar el problema que se desea solucionar. En ambos casos tenemos un problema de clasificación binario, en el primer Dataset debemos identificar billetes falsos. En el segundo Dataset debemos poder afirmar si una oficina se encuentra ocupada o no.

Los atributos del primer Dataset son varianza, skewness, curtosis y entropía.

Los atributos del segundo Dataset son fecha, temperatura, humedad, luz, cantidad de CO2 y la razón de la humedad.

In [2]:
# Cargamos y organizamos el primer Dataset
data_bank = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt', header=None)
data_bank.columns = ["variance", "skewness", "curtosis", "entropy", "class"]

# Remplazamos los 0 por -1 para los modelos
data_bank["class"] = data_bank["class"].replace([0],-1)

# Separamos los atributos de las etiquetas
X_data_bank = data_bank[["variance", "skewness", "curtosis", "entropy"]]
y_data_bank = data_bank["class"]

In [3]:
# Descargamos y descomprimos el segundo Dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00357/occupancy_data.zip"
extract_dir = "occupancy"

# Descomprimimos el zip
zip_path, _ = urllib.request.urlretrieve(url)
with zipfile.ZipFile(zip_path, "r") as f:
    f.extractall(extract_dir)

In [4]:
def occupancy_df_process(path):
  data = pd.read_csv(path, sep=",", 
            names=["col","date","Temperature","Humidity","Light","CO2","HumidityRatio","Occupancy"])[1:]

  data = data.drop(["col"],axis=1)
  # Casteamo todo a float
  data["Light"] = data["Light"].astype(float)
  data["Temperature"] = data["Temperature"].astype(float)
  data["Humidity"] = data["Humidity"].astype(float)
  data["CO2"] = data["CO2"].astype(float)
  data["HumidityRatio"] = data["HumidityRatio"].astype(float)
  data["Occupancy"] = data["Occupancy"].astype(float)


  # Remplazamos los 0 por -1 para los modelos
  data["Occupancy"] = data["Occupancy"].replace([0],-1)

  # Separamos los atributos de las etiquetas
  X_data = data[["Temperature","Humidity","Light","CO2","HumidityRatio"]]
  Y_data = data["Occupancy"]
  return X_data, Y_data

occupancy_train_X , occupancy_train_y = occupancy_df_process("occupancy/datatraining.txt")
occupancy_test_X , occupancy_test_y = occupancy_df_process("occupancy/datatest.txt")
occupancy_test2_X , occupancy_test2_y = occupancy_df_process("occupancy/datatest2.txt")

# Juntamos los dataframes en uno único
occupancy_X = pd.concat([occupancy_train_X,occupancy_test_X, occupancy_test2_X])
occupancy_y = pd.concat([occupancy_train_y, occupancy_test_y, occupancy_test2_y])

## Modelos para el primer Dataset

### SVM

Para el modelo de support vector machine contamos con el hiperparametro de regulación C, el kernel utilizado en el algoritmo, y un parametro que baraja los datos de forma aleatoria. 

Para elegir el mejor valor de "C" se utilizó prueba y error, buscano entre los enteros del 1 al 9.

En el caso del kernel, el algoritmo con el mejor rendimiento fue "RBF" que es una función de base radial utilizada para transformar los datos en un espacio dimensional superior, donde es más probable que los datos sean linealmente separables. La función RBF calcula la similitud entre dos puntos en función de su distancia radial.

In [5]:
sup = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(sup, X_data_bank, y_data_bank, cv=20)
print("Rendimiento en cross val:", scores)
print("Rendimiento overall:", st.mean(scores))

Rendimiento en cross val: [0.97101449 1.         1.         0.98550725 0.98550725 0.97101449
 1.         0.98550725 0.97101449 0.97101449 0.98550725 0.98550725
 1.         1.         1.         1.         1.         0.98529412
 0.98529412 0.98529412]
Rendimiento overall: 0.9883738277919863


In [6]:
sup = svm.SVC(kernel='poly', C=2, random_state=42)
scores = cross_val_score(sup, X_data_bank, y_data_bank, cv=20)
print("Rendimiento en cross val:", scores)
print("Rendimiento overall:", st.mean(scores))

Rendimiento en cross val: [0.95652174 0.98550725 0.97101449 0.98550725 0.97101449 0.95652174
 0.98550725 0.98550725 0.95652174 0.94202899 0.98550725 0.97101449
 0.98529412 0.94117647 1.         1.         0.97058824 0.95588235
 0.95588235 0.98529412]
Rendimiento overall: 0.9723145780051151


In [7]:
sup = svm.SVC(kernel='rbf', C=2, random_state=42)
scores = cross_val_score(sup, X_data_bank, y_data_bank, cv=20)
print("Rendimiento en cross val:", scores)
print("Rendimiento overall:", st.mean(scores))

Rendimiento en cross val: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Rendimiento overall: 1.0


### KNN

El método de clasificación K-Nearest Neighbors (KNN) es un algoritmo de aprendizaje supervisado utilizado para clasificar puntos de datos en función de su cercanía a otros puntos vecinos. Para este modelo tenemos los parametros de la cantidad de neighbors, el peso utilizado en la predicción y el algoritmo para computar los neighbors más cercanos. 

Curiosamente con este modelo la mayoría de hiperparametros generan un rendimiento muy bueno, él que más afecta esto es la cantidad de neighbors. En este primer caso incluso se tiene un muy buen rendimiento con 1 vecindario.

In [8]:
knn = KNeighborsClassifier(n_neighbors = 1, weights="distance", algorithm="ball_tree")
scores = cross_val_score(knn, X_data_bank, y_data_bank, cv=20)
print(scores)
print(st.mean(scores))

[1.         1.         1.         1.         1.         1.
 1.         1.         1.         0.98550725 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.        ]
0.9992753623188406


In [9]:
knn = KNeighborsClassifier(n_neighbors = 10, weights="uniform", algorithm="kd_tree")
scores = cross_val_score(knn, X_data_bank, y_data_bank, cv=20)
print(scores)
print(st.mean(scores))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
1.0


In [10]:
knn = KNeighborsClassifier(n_neighbors = 2, weights="uniform", algorithm="brute")
scores = cross_val_score(knn, X_data_bank, y_data_bank, cv=20)
print(scores)
print(st.mean(scores))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
1.0


### Perceptron

El perceptrón es un modelo de clasificación de datos linealmente separables, almenos cuando se trata de una capa, así que la eficiencia de este método nos permite concluir si el conjunto de datos es linealmente separable.

Los parametros para este modelo son el termino de regularización o penalty, la tolerancia y fit_intercept que es un bool para saber si se debe estimar el intercep. En este caso los mejores para metros son la penalización elasticnet que hace referencia a la norma, calculando el intercept y con una tolerancia de 1e-3.

In [11]:
per = Perceptron(penalty="l2",tol=1e-4,fit_intercept=False, random_state=0)
scores = cross_val_score(per, X_data_bank, y_data_bank, cv=100)
print(scores)
print(st.mean(scores))

[0.85714286 0.92857143 1.         1.         0.92857143 0.92857143
 1.         1.         0.85714286 0.92857143 1.         1.
 1.         1.         0.92857143 0.92857143 1.         0.92857143
 0.92857143 1.         0.85714286 0.85714286 0.92857143 0.92857143
 0.92857143 0.85714286 0.92857143 0.85714286 0.92857143 0.78571429
 1.         0.92857143 1.         1.         1.         0.92857143
 0.85714286 0.78571429 1.         0.85714286 0.92857143 1.
 0.92857143 0.85714286 0.92857143 0.92857143 0.85714286 0.85714286
 1.         0.85714286 0.78571429 0.85714286 0.85714286 1.
 0.92857143 0.92857143 0.85714286 0.85714286 1.         1.
 0.71428571 1.         1.         1.         1.         1.
 0.92857143 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.92307692
 1.         1.         1.         1.         1.         1.
 1.         0.92307692 1.         1.         1.         1.
 0.92307692 0.84615385 0.92307692 0.84615385 0.84615385 0.8

In [12]:
per = Perceptron(penalty="l1",tol=1e-3, fit_intercept=True, random_state=0)
scores = cross_val_score(per, X_data_bank, y_data_bank, cv=100)
print(scores)
print(st.mean(scores))

[0.92857143 1.         0.92857143 1.         1.         0.85714286
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.92857143 1.         1.         1.         1.         1.
 0.85714286 0.92857143 0.92857143 0.85714286 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.92857143 1.         0.92857143 0.92857143 1.
 1.         0.92857143 0.92857143 0.92857143 1.         1.
 1.         1.         1.         0.78571429 1.         1.
 1.         0.92857143 1.         1.         1.         1.
 0.78571429 0.92857143 1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         0.92307692 1.         1.
 1.         1.         1.         0.92307692 1.         0.92307692
 0.92307692 1.         1.         1.    

In [13]:
per = Perceptron(penalty="elasticnet",tol=1e-7,fit_intercept=True, random_state=0)
scores = cross_val_score(per, X_data_bank, y_data_bank, cv=100)
print(scores)
print(st.mean(scores))

[0.92857143 1.         0.92857143 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.92857143 1.         0.92857143 1.         0.92857143
 1.         1.         1.         1.         1.         1.
 1.         0.92857143 1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         0.92857143 0.92857143 1.
 1.         0.92857143 0.92857143 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         0.92857143 0.92857143 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.92307692 1.         1.         1.         1.
 1.         1.         1.         0.92307692 1.         1.
 1.         1.         1.         0.92307692 1.         0.92307692
 0.92307692 1.         1.         1.    

### Logistic

La regresión logística es un algoritmo de aprendizaje supervisado utilizado para problemas de clasificación binaria, es decir, cuando se busca predecir una variable de respuesta que toma dos posibles valores es decir que debería tener un buen desempeño en este caso.

En este modelo no podemos realizar demasiados cambios, estamos sujetos a una penalización l2, y una formulación primal y punicamente el parametro de regularización afecta el resultado, los demás cambios perjudican el modelo radicalmente.

In [14]:
log = LogisticRegression(penalty="l2",dual=False, C=1,random_state=0)
scores = cross_val_score(log, X_data_bank, y_data_bank, cv=100)
print(scores)
print(st.mean(scores))

[0.92857143 1.         0.92857143 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.92857143 1.         1.         1.         1.         1.
 1.         1.         0.92857143 1.         1.         1.
 0.92857143 1.         1.         1.         1.         1.
 1.         0.92857143 1.         0.92857143 0.92857143 1.
 1.         0.92857143 0.92857143 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         0.92857143 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         0.92307692 1.         1.
 1.         1.         1.         0.92307692 1.         0.92307692
 1.         1.         1.         1.        ]
0.

In [15]:
log = LogisticRegression(penalty="l2",dual=False, C=20,random_state=0)
scores = cross_val_score(log, X_data_bank, y_data_bank, cv=100)
print(scores)
print(st.mean(scores))

[0.92857143 1.         0.92857143 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.92857143 1.         1.         1.         1.         1.
 0.92857143 1.         0.92857143 1.         1.         1.
 0.92857143 1.         1.         1.         1.         1.
 1.         0.92857143 1.         0.92857143 0.92857143 1.
 1.         0.92857143 0.92857143 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.92857143 0.92857143 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         0.92307692 1.         1.
 1.         1.         1.         0.92307692 1.         0.92307692
 1.         1.         1.         1.        ]
0.

### Conclusión primer modelo

El rendimiento para este primer Dataset es bastante bueno en todos los modelos en general, sin embargo se deben destacar los modelo de SVM con RBF y el de KNN. Pues muestran resultados perfectos en ambos casos.

## Modelos para el segundo Dataset

### SVM

Para el modelo de support vector machine contamos con el hiperparametro de regulación C, el kernel utilizado en el algoritmo, y un parametro que baraja los datos de forma aleatoria. 

Para elegir el mejor valor de "C" se utilizó prueba y error, buscano entre los enteros del 1 al 9.

En el caso del kernel, el algoritmo con el mejor rendimiento fue entre "poly" y "linear", siendo linear el mejor kernel aunque el entrenamiento de este método es mucho más demorado.

In [16]:
sup2 = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(sup2, occupancy_X, occupancy_y, cv=5)
print(scores)
print(st.mean(scores))

[0.97811284 0.99829767 0.98321984 0.99197471 0.99246109]
0.9888132295719845


In [17]:
sup2 = svm.SVC(kernel='poly',degree=3, C=2, random_state=42)
scores = cross_val_score(sup2, occupancy_X, occupancy_y, cv=5)
print(scores)
print(st.mean(scores))

[0.97811284 0.99708171 0.98273346 0.99294747 0.99002918]
0.9881809338521401


In [18]:
sup2 = svm.SVC(kernel='rbf', C=3, random_state=42)
scores = cross_val_score(sup2, occupancy_X, occupancy_y, cv=5)
print(scores)
print(st.mean(scores))

[0.96449416 0.99878405 0.98078794 0.99270428 0.99124514]
0.985603112840467


### KNN

El método de clasificación K-Nearest Neighbors (KNN) es un algoritmo de aprendizaje supervisado utilizado para clasificar puntos de datos en función de su cercanía a otros puntos vecinos. Para este modelo tenemos los parametros de la cantidad de neighbors, el peso utilizado en la predicción y el algoritmo para computar los neighbors más cercanos. 

En este modelo importa aún más la cantidad de vecindades, así que iterando por los valores de entre 1 a 100 estos valores reflejan el mejor comportamiento, el peso uniforme refleja tambien un mejor rendimiento y además el algoritmo kd_tree.

In [19]:
knn2 = KNeighborsClassifier(n_neighbors = 13, weights="distance", algorithm="ball_tree")
scores = cross_val_score(knn2, occupancy_X, occupancy_y, cv=100)
print(scores)
print(st.mean(scores))

[0.98058252 0.99514563 1.         1.         1.         0.68932039
 1.         0.99029126 1.         1.         0.94660194 0.9368932
 1.         0.99514563 1.         1.         1.         0.97087379
 0.88349515 1.         1.         1.         1.         0.99514563
 0.99029126 1.         1.         1.         1.         0.99514563
 1.         1.         1.         1.         0.96601942 1.
 1.         1.         1.         1.         0.85436893 1.
 0.99029126 1.         1.         0.95145631 0.97572816 0.95145631
 1.         1.         1.         0.90776699 0.98543689 0.97572816
 0.96601942 1.         0.97572816 0.98058252 1.         1.
 0.9902439  1.         1.         0.9902439  0.90243902 1.
 0.9902439  0.96097561 1.         1.         1.         0.93658537
 1.         1.         1.         1.         0.97560976 1.
 0.9902439  0.99512195 0.96585366 1.         1.         1.
 1.         1.         1.         1.         0.89756098 1.
 1.         1.         1.         0.99512195 0.97073

In [20]:
knn2 = KNeighborsClassifier(n_neighbors = 25, weights="uniform", algorithm="kd_tree")
scores = cross_val_score(knn2, occupancy_X, occupancy_y, cv=100)
print(scores)
print(st.mean(scores))

[0.99514563 1.         1.         1.         1.         0.68932039
 1.         1.         1.         1.         0.94660194 0.95145631
 1.         1.         1.         1.         1.         0.97572816
 1.         1.         1.         1.         1.         0.99514563
 1.         1.         1.         1.         1.         0.99514563
 1.         1.         1.         1.         0.98058252 1.
 1.         1.         1.         1.         0.8592233  1.
 1.         1.         1.         0.94660194 0.97572816 1.
 1.         1.         1.         0.90776699 0.99514563 1.
 1.         1.         0.98058252 0.98543689 1.         1.
 0.9902439  1.         1.         0.9902439  0.90243902 1.
 0.9902439  0.9902439  1.         1.         1.         0.98536585
 1.         1.         1.         1.         0.99512195 1.
 1.         0.99512195 0.92195122 1.         1.         1.
 1.         1.         1.         1.         0.93658537 1.
 1.         1.         1.         0.99512195 0.97073171 1.
 1.     

In [21]:
knn2 = KNeighborsClassifier(n_neighbors = 50, weights="uniform", algorithm="brute")
scores = cross_val_score(knn2, occupancy_X, occupancy_y, cv=100)
print(scores)
print(st.mean(scores))

[0.99514563 1.         1.         1.         1.         0.68932039
 0.99514563 1.         1.         1.         0.94660194 0.96116505
 1.         1.         1.         1.         1.         0.97572816
 1.         1.         1.         1.         1.         0.99514563
 1.         1.         1.         1.         1.         0.99514563
 1.         1.         1.         1.         0.98058252 1.
 1.         1.         1.         1.         0.86893204 1.
 1.         1.         1.         0.94660194 0.97572816 1.
 1.         1.         1.         0.90776699 0.99514563 1.
 1.         1.         0.98058252 0.98058252 1.         1.
 0.9902439  1.         1.         0.9902439  0.90243902 1.
 0.9902439  1.         1.         1.         1.         0.98536585
 1.         1.         1.         1.         0.99512195 1.
 1.         0.9902439  0.87317073 1.         1.         1.
 1.         1.         1.         1.         0.94146341 1.
 1.         1.         1.         0.99512195 0.97073171 1.
 1.     

###Perceptron

El perceptrón es un modelo de clasificación de datos linealmente separables, almenos cuando se trata de una capa, así que la eficiencia de este método nos permite concluir si el conjunto de datos es linealmente separable.

Los parametros para este modelo son el termino de regularización o penalty, la tolerancia y fit_intercept que es un bool para saber si se debe estimar el intercep. En este caso los mejores para metros son la penalización l1 que hace referencia a la norma, calculando el intercept y con una tolerancia de 1e-3.

In [22]:
per2 = Perceptron(penalty="l2",tol=1e-3, fit_intercept=False, random_state=0)
scores = cross_val_score(per2, occupancy_X, occupancy_y, cv=100)
print(scores)
print(st.mean(scores))

[1.         0.76699029 0.99514563 1.         1.         0.68932039
 1.         1.         1.         1.         0.70873786 0.88834951
 0.76699029 1.         1.         1.         0.95145631 0.23300971
 0.65533981 1.         1.         1.         0.76699029 1.
 0.98058252 1.         0.99514563 0.5631068  1.         0.76699029
 1.         0.76699029 0.76699029 0.76699029 0.77669903 0.76699029
 1.         1.         1.         1.         0.87378641 1.
 1.         0.76699029 1.         0.94660194 0.97087379 1.
 1.         1.         0.77184466 0.90776699 1.         1.
 1.         1.         0.76699029 0.98058252 0.99514563 1.
 1.         1.         1.         1.         0.8097561  1.
 1.         1.         1.         1.         1.         0.95609756
 1.         1.         1.         1.         0.99512195 0.77073171
 1.         0.93170732 0.4        1.         1.         1.
 1.         1.         1.         1.         0.94146341 1.
 1.         1.         1.         0.99512195 0.97073171 0.7

In [23]:
per2 = Perceptron(penalty="l1",tol=1e-3, fit_intercept=True, random_state=0)
scores = cross_val_score(per2, occupancy_X, occupancy_y, cv=100)
print(scores)
print(st.mean(scores))

[1.         1.         1.         1.         1.         0.68932039
 0.94174757 1.         1.         1.         0.94660194 0.96601942
 1.         1.         1.         1.         1.         0.41262136
 1.         1.         1.         1.         1.         0.99514563
 1.         1.         1.         1.         1.         0.76699029
 1.         0.76699029 1.         0.76699029 0.98543689 0.88349515
 1.         1.         1.         1.         0.87864078 1.
 1.         1.         1.         0.95145631 0.97572816 1.
 1.         1.         1.         0.90776699 0.99514563 1.
 1.         1.         0.98058252 0.83980583 0.77184466 1.
 0.99512195 1.         1.         0.9902439  0.90243902 1.
 0.9902439  1.         1.         1.         1.         0.97560976
 1.         1.         1.         1.         0.99512195 1.
 1.         1.         0.8195122  1.         1.         1.
 1.         1.         1.         1.         0.93658537 1.
 1.         1.         1.         0.99512195 0.97073171 1.


In [24]:
per2 = Perceptron(penalty="elasticnet",tol=1e-7,fit_intercept=True, random_state=0)
scores = cross_val_score(per2, occupancy_X, occupancy_y, cv=100)
print(scores)
print(st.mean(scores))

[1.         1.         1.         1.         1.         0.54368932
 1.         1.         0.76699029 1.         0.88349515 0.96601942
 1.         1.         1.         1.         0.98543689 0.23300971
 0.62135922 1.         1.         1.         1.         1.
 1.         0.91747573 0.76699029 0.76699029 0.80097087 0.76699029
 1.         0.83009709 0.83009709 0.76699029 0.97572816 1.
 1.         1.         1.         1.         0.87378641 1.
 1.         1.         1.         0.90291262 0.97572816 1.
 1.         0.76699029 1.         0.90776699 0.82038835 1.
 1.         1.         0.84466019 0.94174757 1.         1.
 1.         1.         1.         0.99512195 0.83902439 1.
 1.         1.         1.         1.         1.         0.88292683
 1.         1.         1.         1.         0.99512195 1.
 1.         0.76097561 0.43902439 1.         0.22926829 0.9804878
 1.         1.         0.77073171 1.         0.94146341 1.
 1.         0.77073171 1.         0.99512195 0.9804878  0.99512195
 

### Logistic

La regresión logística es un algoritmo de aprendizaje supervisado utilizado para problemas de clasificación binaria, es decir, cuando se busca predecir una variable de respuesta que toma dos posibles valores es decir que debería tener un buen desempeño en este caso.

En este modelo no podemos realizar demasiados cambios, estamos sujetos a una penalización l2, y una formulación primal y punicamente el parametro de regularización afecta el resultado, los demás cambios perjudican el modelo radicalmente.

In [25]:
log2 = LogisticRegression(penalty="l2",dual=False, C=1,random_state=0)
scores = cross_val_score(log2, occupancy_X, occupancy_y, cv=100)
print(scores)
print(st.mean(scores))

[0.99514563 1.         1.         1.         1.         0.68932039
 0.99029126 1.         1.         1.         0.94660194 0.96601942
 1.         1.         1.         1.         1.         0.97572816
 1.         1.         1.         1.         1.         0.99514563
 1.         1.         1.         0.99514563 1.         0.99514563
 1.         1.         1.         1.         0.98543689 1.
 1.         1.         1.         1.         0.86893204 1.
 1.         1.         1.         0.95145631 0.97572816 1.
 1.         1.         1.         0.90776699 0.99514563 1.
 1.         1.         0.98058252 0.98543689 1.         1.
 0.99512195 1.         1.         0.9902439  0.89756098 1.
 0.9902439  1.         1.         1.         1.         0.9804878
 1.         1.         1.         1.         0.99512195 1.
 1.         0.9804878  0.95121951 1.         1.         1.
 1.         1.         1.         1.         0.94634146 1.
 1.         1.         1.         0.99512195 0.97073171 1.
 1.      

In [26]:
log2 = LogisticRegression(penalty="l2",dual=False, C=50,random_state=0)
scores = cross_val_score(log2, occupancy_X, occupancy_y, cv=100)
print(scores)
print(st.mean(scores))

[0.99514563 1.         1.         1.         1.         0.68932039
 0.99029126 1.         1.         1.         0.94660194 0.96601942
 1.         1.         1.         1.         1.         0.97572816
 1.         1.         1.         1.         1.         0.99514563
 1.         1.         1.         0.99514563 1.         0.99514563
 1.         1.         1.         1.         0.98543689 1.
 1.         1.         1.         1.         0.86893204 1.
 1.         1.         1.         0.95145631 0.97572816 1.
 1.         1.         1.         0.90776699 0.99514563 1.
 1.         1.         0.98058252 0.98543689 1.         1.
 0.99512195 1.         1.         0.98536585 0.89756098 1.
 0.9902439  1.         1.         1.         1.         0.9804878
 1.         1.         1.         1.         0.99512195 1.
 1.         0.9804878  0.95121951 1.         1.         1.
 1.         1.         1.         1.         0.94146341 1.
 1.         1.         1.         0.99512195 0.97073171 1.
 1.      

### Conclusión segundo Dataset

Para el segundo Dataset la mayoría de modelos tiene un éxtio del 98%, se destacan en particular SVM y Logistic.